In [18]:
# Import libraries and load datasets
import os
import re
import unicodedata
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
paths = {
    'symptoms': 'symptoms.csv',
    'allergies': 'allergies.csv',
    'careplans': 'careplans.csv',
    'conditions': 'conditions.csv',
    'immunizations': 'immunizations.csv',
    'observations': 'observations.csv',
    'procedures': 'procedures.csv',
    'medicine': 'medicine_disease.csv',
    'herbal': 'herbaltreatment_disease.csv',
    'nutrition': 'nutrition_disease.csv',
}

datasets = {}
for name, p in paths.items():
    if os.path.exists(p):
        try:
            datasets[name] = pd.read_csv(p)
            print(f"Loaded {name}: {datasets[name].shape[0]} rows from {p}")
        except Exception as e:
            print(f"Failed to load {name} from {p}: {e}")
    else:
        print(f"Warning: {name} file not found at {p}")

Loaded symptoms: 1000 rows from symptoms.csv
Loaded allergies: 600 rows from allergies.csv
Loaded careplans: 600 rows from careplans.csv
Loaded conditions: 600 rows from conditions.csv
Loaded immunizations: 600 rows from immunizations.csv
Loaded observations: 600 rows from observations.csv
Loaded procedures: 600 rows from procedures.csv
Loaded medicine: 300 rows from medicine_disease.csv
Loaded herbal: 300 rows from herbaltreatment_disease.csv
Loaded nutrition: 300 rows from nutrition_disease.csv


In [19]:
# Text preprocessing
def clean_text(s):
    if not isinstance(s, str):
        s = str(s)
    s = s.lower()
    s = unicodedata.normalize("NFKD", s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

In [29]:
#  Build matching corpus 
corpus = []

# === PRIORITY 1: SYMPTOMS.CSV (Primary - Most Valuable) ===
if 'symptoms' in datasets:
    sym_df = datasets['symptoms']
    disease_col = 'Disease'
    symptoms_col = 'Symptoms'
    
    symptom_entries = 0
    for i, r in sym_df.iterrows():
        disease = str(r.get(disease_col, '')).strip()
        symptoms = str(r.get(symptoms_col, '')).strip()
        
        if disease and symptoms:
            # Clean and format symptoms
            clean_symptoms = symptoms.replace(';', ' ').replace(',', ' ')
            text = f"symptoms: {clean_symptoms}"
            corpus.append({
                'id': f'sym_{i}', 
                'type': 'symptoms', 
                'text': text, 
                'diagnosis': disease,
                'symptoms': clean_symptoms,
                'priority': 1
            })
            symptom_entries += 1
    print(f"✓ Added {symptom_entries} symptom-disease mappings")

# === PRIORITY 2: OBSERVATIONS (Clinical Context) ===
if 'observations' in datasets:
    obs_df = datasets['observations']
    obs_col = next((c for c in obs_df.columns if 'obser' in c.lower() or 'symptom' in c.lower() or 'description' in c.lower()), obs_df.columns[0])
    cause_col = next((c for c in obs_df.columns if 'cause' in c.lower() or 'possible' in c.lower() or 'diagnosis' in c.lower()), None)
    
    observation_entries = 0
    for i, r in obs_df.iterrows():
        obs = str(r.get(obs_col, '')).strip()
        cause = str(r.get(cause_col, '')).strip() if cause_col else ''
        
        if obs:
            text = f"observation: {obs}"
            if cause:
                text += f" possible_cause: {cause}"
            
            diagnosis = cause if cause else obs
            corpus.append({
                'id': f'obs_{i}', 
                'type': 'observation', 
                'text': text, 
                'diagnosis': diagnosis,
                'observation': obs,
                'cause': cause,
                'priority': 2
            })
            observation_entries += 1
    print(f"✓ Added {observation_entries} clinical observations")

# === PRIORITY 3: CONDITIONS (Structured Medical Knowledge) ===
if 'conditions' in datasets:
    cond_df = datasets['conditions']
    cond_col = next((c for c in cond_df.columns if 'cond' in c.lower() or 'condition' in c.lower() or 'diagnosis' in c.lower()), cond_df.columns[0])
    
    # Look for additional context columns
    desc_col = next((c for c in cond_df.columns if 'desc' in c.lower() or 'description' in c.lower()), None)
    severity_col = next((c for c in cond_df.columns if 'sever' in c.lower()), None)
    age_col = next((c for c in cond_df.columns if 'age' in c.lower()), None)
    gender_col = next((c for c in cond_df.columns if 'gender' in c.lower()), None)
    
    condition_entries = 0
    for i, r in cond_df.iterrows():
        condition = str(r.get(cond_col, '')).strip()
        description = str(r.get(desc_col, '')).strip() if desc_col else ''
        severity = str(r.get(severity_col, '')).strip() if severity_col else ''
        age = str(r.get(age_col, '')).strip() if age_col else ''
        gender = str(r.get(gender_col, '')).strip() if gender_col else ''
        
        if condition:
            text_parts = [f"condition: {condition}"]
            if description:
                text_parts.append(f"description: {description}")
            if severity:
                text_parts.append(f"severity: {severity}")
            if age:
                text_parts.append(f"age: {age}")
            if gender:
                text_parts.append(f"gender: {gender}")
            
            text = " ".join(text_parts)
            corpus.append({
                'id': f'cond_{i}', 
                'type': 'condition', 
                'text': text, 
                'diagnosis': condition,
                'description': description,
                'priority': 3
            })
            condition_entries += 1
    print(f"✓ Added {condition_entries} medical conditions")

# === PRIORITY 4: CAREPLANS (Treatment Context) ===
if 'careplans' in datasets:
    cp_df = datasets['careplans']
    diag_col = next((c for c in cp_df.columns if 'diag' in c.lower() or 'condition' in c.lower()), cp_df.columns[0])
    plan_col = next((c for c in cp_df.columns if 'care' in c.lower() or 'plan' in c.lower() or 'treatment' in c.lower()), None)
    goal_col = next((c for c in cp_df.columns if 'goal' in c.lower() or 'objective' in c.lower()), None)
    
    careplan_entries = 0
    for i, r in cp_df.iterrows():
        diag = str(r.get(diag_col, '')).strip()
        plan = str(r.get(plan_col, '')).strip() if plan_col else ''
        goal = str(r.get(goal_col, '')).strip() if goal_col else ''
        
        if diag:
            text_parts = [f"diagnosis: {diag}"]
            if plan:
                text_parts.append(f"care_plan: {plan}")
            if goal:
                text_parts.append(f"treatment_goal: {goal}")
            
            text = " ".join(text_parts)
            corpus.append({
                'id': f'cp_{i}', 
                'type': 'careplan', 
                'text': text, 
                'diagnosis': diag,
                'care_plan': plan,
                'priority': 4
            })
            careplan_entries += 1
    print(f"✓ Added {careplan_entries} care plans")

# === PRIORITY 5: ALLERGIES (Additional Medical Context) ===
if 'allergies' in datasets:
    allergy_df = datasets['allergies']
    allergy_col = next((c for c in allergy_df.columns if 'aller' in c.lower() or 'reaction' in c.lower()), allergy_df.columns[0])
    severity_col = next((c for c in allergy_df.columns if 'sever' in c.lower()), None)
    
    allergy_entries = 0
    for i, r in allergy_df.iterrows():
        allergy = str(r.get(allergy_col, '')).strip()
        severity = str(r.get(severity_col, '')).strip() if severity_col else ''
        
        if allergy:
            text = f"allergy: {allergy}"
            if severity:
                text += f" severity: {severity}"
            
            corpus.append({
                'id': f'allergy_{i}', 
                'type': 'allergy', 
                'text': text, 
                'diagnosis': f"Allergy: {allergy}",
                'priority': 5
            })
            allergy_entries += 1
    print(f"✓ Added {allergy_entries} allergy entries")

# === DEDUPLICATION AND QUALITY CONTROL ===
print("\n🔍 Performing corpus optimization...")

# Remove exact duplicates
initial_count = len(corpus)
seen_texts = set()
unique_corpus = []

for item in corpus:
    clean_t = clean_text(item['text'])
    if clean_t not in seen_texts and clean_t.strip():
        seen_texts.add(clean_t)
        unique_corpus.append(item)

corpus = unique_corpus
print(f"Removed {initial_count - len(corpus)} duplicates")

# Sort by priority (higher priority first)
corpus.sort(key=lambda x: x.get('priority', 999))

# === FALLBACK: Ensure we always have some data ===
if len(corpus) == 0:
    print("⚠️  No datasets found - creating fallback entries")
    corpus = [
        {
            'id': 'fallback_1', 'type': 'fallback', 'priority': 999,
            'text': 'symptoms: fever cough headache fatigue', 
            'diagnosis': 'Viral Infection'
        },
        {
            'id': 'fallback_2', 'type': 'fallback', 'priority': 999,
            'text': 'symptoms: sore throat runny nose sneezing', 
            'diagnosis': 'Common Cold'
        },
        {
            'id': 'fallback_3', 'type': 'fallback', 'priority': 999,
            'text': 'symptoms: chest pain shortness of breath', 
            'diagnosis': 'Respiratory Issue'
        },
    ]

# === FINAL CORPUS STATISTICS ===
print(f"\n🎯 FINAL CORPUS BUILT: {len(corpus)} total entries")
type_counts = {}
for item in corpus:
    t = item['type']
    type_counts[t] = type_counts.get(t, 0) + 1

print("Corpus composition:")
for type_name, count in type_counts.items():
    print(f"  {type_name}: {count} entries")

# Create DataFrame and prepare for TF-IDF
corpus_df = pd.DataFrame(corpus)
corpus_df['clean_text'] = corpus_df['text'].apply(clean_text)

print(f"\n🧠 Training TF-IDF model on {len(corpus_df)} unique entries...")
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', min_df=2)
tfidf_matrix = vectorizer.fit_transform(corpus_df['clean_text'])
print(f"✓ TF-IDF vocabulary size: {len(vectorizer.vocabulary_)} terms")

✓ Added 1000 symptom-disease mappings
✓ Added 600 clinical observations
✓ Added 600 medical conditions
✓ Added 600 care plans
✓ Added 600 allergy entries

🔍 Performing corpus optimization...
Removed 1851 duplicates

🎯 FINAL CORPUS BUILT: 1549 total entries
Corpus composition:
  symptoms: 1000 entries
  observation: 8 entries
  condition: 525 entries
  careplan: 8 entries
  allergy: 8 entries

🧠 Training TF-IDF model on 1549 unique entries...
✓ TF-IDF vocabulary size: 1453 terms


In [30]:
#  Diagnosis matcher function
def match_diagnoses(symptom_text, top_k=5):
    s = clean_text(symptom_text)
    if not s or tfidf_matrix is None:
        return []
    v = vectorizer.transform([s])
    sims = cosine_similarity(v, tfidf_matrix).flatten()
    score_by_diag = defaultdict(float)
    count_by_diag = defaultdict(int)
    for idx, sim in enumerate(sims):
        diag = corpus_df.loc[idx, 'diagnosis'] or 'Unknown'
        score_by_diag[diag] += float(sim)
        count_by_diag[diag] += 1
    results = []
    for diag, score in score_by_diag.items():
        avg_score = score / max(1, count_by_diag[diag])
        results.append((diag, avg_score))
    results.sort(key=lambda x: x[1], reverse=True)
    filtered = [r for r in results if r[1] > 0.01]
    return filtered[:top_k]

In [22]:
# Cell 5: Lookup and recommendation functions
def lookup_table(df, key_col, value_col, diagnosis):
    if df is None or diagnosis is None:
        return []
    
    diag_l = diagnosis.strip().lower()
    results = []
    
    # Try exact match first
    if key_col in df.columns:
        exact_matches = df[df[key_col].astype(str).str.lower() == diag_l]
        if len(exact_matches) > 0:
            results.extend(exact_matches[value_col].dropna().astype(str).tolist())
    
    # Try partial matches if no exact matches found
    if not results and key_col in df.columns:
        # Split diagnosis into words and try to match any part
        diag_words = diag_l.split()
        for word in diag_words:
            if len(word) > 3:  # Only search for words longer than 3 characters
                contains_matches = df[df[key_col].astype(str).str.lower().str.contains(word, na=False)]
                if len(contains_matches) > 0:
                    results.extend(contains_matches[value_col].dropna().astype(str).tolist())
    
    # Fallback: search any column for the diagnosis
    if not results:
        for c in df.columns:
            if c != value_col:  # Don't search the value column itself
                subset = df[df[c].astype(str).str.lower().str.contains(diag_l, na=False)]
                if len(subset) > 0:
                    results.extend(subset[value_col].dropna().astype(str).tolist())
                    break
    
    # Return unique results
    return list(dict.fromkeys(results))

def recommend_medicines(diagnosis, top_n=5):
    df = datasets.get('medicine')
    if df is None:
        print("Medicine dataset not available")
        return []
    
    # More flexible column detection
    key_col = next((c for c in df.columns if any(x in c.lower() for x in ['disease', 'condition', 'diagnosis', 'illness', 'disorder'])), df.columns[0])
    val_col = next((c for c in df.columns if any(x in c.lower() for x in ['medicine', 'medication', 'drug', 'treatment', 'prescription', 'therapy'])), None)
    
    if val_col is None:
        # If no obvious value column, use the second column or first if only one column
        val_col = df.columns[1] if df.shape[1] > 1 else df.columns[0]
    
    print(f"Medicine lookup: diagnosis='{diagnosis}', key_col='{key_col}', val_col='{val_col}'")
    results = lookup_table(df, key_col, val_col, diagnosis)
    print(f"Found medicine results: {results}")
    return results[:top_n]

def recommend_herbal(diagnosis, top_n=5):
    df = datasets.get('herbal')
    if df is None:
        print("Herbal dataset not available")
        return []
    
    key_col = next((c for c in df.columns if any(x in c.lower() for x in ['disease', 'condition', 'diagnosis'])), df.columns[0])
    val_col = next((c for c in df.columns if any(x in c.lower() for x in ['herbal', 'remedy', 'treatment', 'plant', 'natural'])), None)
    
    if val_col is None:
        val_col = df.columns[1] if df.shape[1] > 1 else df.columns[0]
    
    print(f"Herbal lookup: diagnosis='{diagnosis}', key_col='{key_col}', val_col='{val_col}'")
    results = lookup_table(df, key_col, val_col, diagnosis)
    print(f"Found herbal results: {results}")
    return results[:top_n]

def recommend_nutrition(diagnosis, top_n=5):
    df = datasets.get('nutrition')
    if df is None:
        print("Nutrition dataset not available")
        return []
    
    key_col = next((c for c in df.columns if any(x in c.lower() for x in ['disease', 'condition', 'diagnosis'])), df.columns[0])
    val_col = next((c for c in df.columns if any(x in c.lower() for x in ['nutrition', 'food', 'diet', 'supplement', 'vitamin'])), None)
    
    if val_col is None:
        val_col = df.columns[1] if df.shape[1] > 1 else df.columns[0]
    
    print(f"Nutrition lookup: diagnosis='{diagnosis}', key_col='{key_col}', val_col='{val_col}'")
    results = lookup_table(df, key_col, val_col, diagnosis)
    print(f"Found nutrition results: {results}")
    return results[:top_n]

In [31]:
# Cell 6: Lifestyle recommendations functions
def extract_lifestyle_from_careplan(careplan_text):
    lifestyle_terms = []
    text_lower = careplan_text.lower()
    
    if 'diet' in text_lower:
        if 'modification' in text_lower:
            lifestyle_terms.append('Follow dietary modifications as prescribed')
        else:
            lifestyle_terms.append('Maintain healthy diet')
    
    if 'exercise' in text_lower or 'physical activity' in text_lower:
        lifestyle_terms.append('Regular physical activity as recommended')
    
    if 'rehabilitation' in text_lower:
        lifestyle_terms.append('Participate in recommended rehabilitation program')
    
    if 'therapy' in text_lower and 'behavioral' in text_lower:
        lifestyle_terms.append('Engage in behavioral therapy sessions')
    
    if 'monitoring' in text_lower:
        lifestyle_terms.append('Regular self-monitoring as advised')
    
    return lifestyle_terms if lifestyle_terms else ['Follow prescribed care plan']

def get_generic_lifestyle_recommendations(diagnosis):
    diag = (diagnosis or '').lower()
    recs = []
    
    if any(x in diag for x in ['migraine', 'headache']):
        recs += [
            'Identify and avoid headache triggers',
            'Maintain regular sleep schedule',
            'Stay hydrated',
            'Manage stress through relaxation techniques'
        ]
    elif 'hypertension' in diag or 'high blood pressure' in diag:
        recs += [
            'Reduce salt intake',
            'Maintain healthy weight',
            'Limit alcohol consumption',
            'Regular aerobic exercise'
        ]
    elif 'diabetes' in diag:
        recs += [
            'Monitor blood glucose regularly',
            'Reduce refined carbohydrates and sugary drinks',
            'Increase physical activity'
        ]
    elif 'asthma' in diag:
        recs += [
            'Avoid known triggers (smoke, pollen, dust)',
            'Follow inhaler/medication plan',
            'Consider allergen-proofing home'
        ]
    elif 'heart' in diag:
        recs += [
            'Cardiac-friendly diet low in saturated fats',
            'Regular moderate exercise',
            'Stress management techniques'
        ]
    elif 'kidney' in diag:
        recs += [
            'Follow renal diet restrictions',
            'Monitor fluid intake',
            'Regular kidney function tests'
        ]
    elif 'obesity' in diag:
        recs += [
            'Calorie-controlled balanced diet',
            'Regular physical activity',
            'Behavior modification for eating habits'
        ]
    elif 'depression' in diag:
        recs += [
            'Regular sleep schedule',
            'Social engagement and support',
            'Mindfulness and relaxation techniques'
        ]
    elif 'infection' in diag or 'fever' in diag:
        recs += [
            'Stay hydrated',
            'Rest and seek medical evaluation if symptoms worsen'
        ]
    
    general_recs = ['Maintain balanced diet', 'Regular exercise', 'Get routine checkups']
    all_recs = recs + general_recs
    return list(dict.fromkeys(all_recs))[:5]

def lifestyle_recommendations(diagnosis):
    if 'careplans' not in datasets:
        return get_generic_lifestyle_recommendations(diagnosis)
    
    cp_df = datasets['careplans']
    diag_col = next((c for c in cp_df.columns if 'diag' in c.lower()), cp_df.columns[0])
    plan_col = next((c for c in cp_df.columns if 'care' in c.lower() or 'plan' in c.lower()), None)
    
    if plan_col is None:
        return get_generic_lifestyle_recommendations(diagnosis)
    
    diag_l = (diagnosis or '').strip().lower()
    
    # Try exact match first
    exact_matches = cp_df[cp_df[diag_col].astype(str).str.lower() == diag_l]
    if len(exact_matches) > 0:
        plans = exact_matches[plan_col].dropna().astype(str).tolist()
        if plans:
            lifestyle_advice = []
            for plan in plans:
                lifestyle_advice.extend(extract_lifestyle_from_careplan(plan))
            
            if lifestyle_advice:
                return list(dict.fromkeys(lifestyle_advice))[:5]
    
    # Try partial match
    for word in diag_l.split():
        if len(word) > 4:
            partial_matches = cp_df[cp_df[diag_col].astype(str).str.lower().str.contains(word, na=False)]
            if len(partial_matches) > 0:
                plans = partial_matches[plan_col].dropna().astype(str).tolist()
                if plans:
                    lifestyle_advice = []
                    for plan in plans:
                        lifestyle_advice.extend(extract_lifestyle_from_careplan(plan))
                    
                    if lifestyle_advice:
                        return list(dict.fromkeys(lifestyle_advice))[:5]
    
    return get_generic_lifestyle_recommendations(diagnosis)

In [32]:
# Cell 7: Compose advice function
def compose_advice(diagnosis, score):
    meds = recommend_medicines(diagnosis)
    herbs = recommend_herbal(diagnosis)
    nuts = recommend_nutrition(diagnosis)
    life = lifestyle_recommendations(diagnosis)
    lines = []
    lines.append(f"Most likely diagnosis: {diagnosis} (score={score:.3f})")
    if meds:
        lines.append("Recommended medicines: " + "; ".join(meds))
    if herbs:
        lines.append("Suggested herbal treatments: " + "; ".join(herbs))
    if nuts:
        lines.append("Nutrition advice: " + "; ".join(nuts))
    lines.append("Lifestyle recommendations: " + "; ".join(life))
    return "\n".join(lines)

In [33]:
# Cell 8: Interactive advisor function
def run_advisor_interactive():
    print("=== Health Advisor ===")
    name = input("Patient name (optional): ").strip()
    age = input("Age (optional): ").strip()
    gender = input("Gender (optional): ").strip()
    symptoms = input("Describe symptoms and signs (brief): ").strip()
    if not symptoms:
        print("No symptoms provided. Exiting.")
        return
    
    context_parts = [symptoms]
    if age:
        context_parts.append("age " + age)
    if gender:
        context_parts.append("gender " + gender)
    
    context = " ".join(context_parts)

    matches = match_diagnoses(context, top_k=5)
    if not matches:
        print("No probable diagnosis found. Consider consulting a clinician.")
        return

    composed = []
    for diag, score in matches[:3]:
        composed.append(compose_advice(diag, score))
    final_text = "\n\n---\n\n".join(composed)

    print("\n===== HEALTH ADVICE =====\n")
    print(final_text)

print("\nReady. To use the advisor interactively, run:\n\nrun_advisor_interactive()\n")


Ready. To use the advisor interactively, run:

run_advisor_interactive()



In [38]:
run_advisor_interactive()

=== Health Advisor ===


Patient name (optional):  he
Age (optional):  
Gender (optional):  
Describe symptoms and signs (brief):  headache


Medicine lookup: diagnosis='Hyperthyroidism (chronic)', key_col='Disease', val_col='Medicine'
Found medicine results: []
Herbal lookup: diagnosis='Hyperthyroidism (chronic)', key_col='Disease', val_col='Herb'
Found herbal results: []
Nutrition lookup: diagnosis='Hyperthyroidism (chronic)', key_col='Disease', val_col='Recommended_Food'
Found nutrition results: []
Medicine lookup: diagnosis='Migraine or hypertension', key_col='Disease', val_col='Medicine'
Found medicine results: ['Ranitidine', 'Hydroxychloroquine', 'Paracetamol', 'Omeprazole', 'Diazepam', 'Ciprofloxacin', 'Hydrochlorothiazide', 'Azithromycin', 'Salbutamol', 'Chloroquine', 'Metformin', 'Aspirin', 'Ibuprofen', 'Metronidazole', 'Insulin']
Herbal lookup: diagnosis='Migraine or hypertension', key_col='Disease', val_col='Herb'
Found herbal results: ['Garlic', 'Ashwagandha', 'Licorice Root', 'Ginger Root', 'Sage', 'Turmeric', 'Coriander', 'Lemongrass', 'Basil', 'Holy Basil', 'Neem']
Nutrition lookup: diagnosis='Migraine or hype

/tmp/ipykernel_14307/18853937.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_matches = df[df[key_col].astype(str).str.lower().str.contains(word, na=False)]
/tmp/ipykernel_14307/18853937.py:29: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset = df[df[c].astype(str).str.lower().str.contains(diag_l, na=False)]
/tmp/ipykernel_14307/18853937.py:29: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset = df[df[c].astype(str).str.lower().str.contains(diag_l, na=False)]
/tmp/ipykernel_14307/18853937.py:29: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset = df[df[c].astype(str).str.lower().str.contains(diag_l, na=False)]
/t